<a href="https://colab.research.google.com/github/osa-adler/big-data-challenge/blob/main/Digital_VideoGames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install --only-upgrade openjdk-11-jdk-headless -qq > /dev/null
!wget -q -c http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 https://develo

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("bigdata-homework1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
digital_video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
digital_video_games_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...|2015-08-31 00:00:00|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live

In [4]:
# Count the number of records (rows) in the dataset
print(digital_video_games_df.count())
digital_video_games_df = digital_video_games_df.dropna()
print(digital_video_games_df.count())
digital_video_games_df= digital_video_games_df.dropDuplicates()
print(digital_video_games_df.count())

145431
145425
145425


In [5]:
# create the review_id dataframe for the 
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = digital_video_games_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RDHZ0IP0QWF3G|   39005147|B00SLM597S|     500010339| 2015-08-31|
|R2AQD17WE4BPXV|   10726509|B009IR1SR0|     912779847| 2015-08-20|
|R27V3DHADX5APY|    1709090|B00F4CEHNK|     341969535| 2015-08-19|
|R2Q0PGNE0M3ES4|     167253|B00TUEOSMS|      47086315| 2015-08-18|
|R1LB0DS6OZCIMW|   10534940|B00TUEOSMS|      47086315| 2015-08-17|
|R3HX7QICC0NOCK|   11267480|B00GAC1D2G|     384246568| 2015-08-15|
|R1KXTQXBE0CRQM|   22661511|B00T8NURF8|     181827382| 2015-08-15|
|R39TG3LST9JDA8|   18404333|B002VBWIP6|     232803743| 2015-08-10|
|R1W2UR1JVB2C13|   22353312|B010R8DUFQ|     473214709| 2015-08-09|
|R1M0DM7FCXI1H9|     535447|B00GAC1D2G|     384246568| 2015-08-06|
|R2IJD4JDICI4AF|   10733761|B009IR1SR0|     912779847| 2015-08-01|
| RWVPFDHF1M46B|    2585643|B00GAC1D2G|     384246568| 2015-07

In [6]:
# Products DataFrame
products_df = digital_video_games_df.select(["product_id", "product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001TK3VTC|Cubis Gold 2 [Dow...|
|B002MUB0TG|Little Shop of Tr...|
|B00DRO824C|X2 The Threat [On...|
|B00CMDKNHI| 100% Hidden Objects|
|B00CMSCTA2|Age of Empires II...|
|B00GDHD3LW|1 Month Subscript...|
|B00OM1VVJM|            NBa 2K15|
|B005XOJ6NK|   Sonic Generations|
|B00GGUHGGY|Call of Duty Blac...|
|B00A9XQ890|Grand Theft Auto:...|
|B003YMMGEE|Sniper: Ghost War...|
|B00405RA04|Rome: Total War -...|
|B0083KX9IO|Stronghold Kingdo...|
|B00M0W1YZQ|Chronicles of Wit...|
|B00895ARJC|  Syberia [Download]|
|B0084FHQ1O|The Stronghold Co...|
|B00KK9PR90|Freddi Fish 3: Th...|
|B001KC01OW|Wonderlines [Down...|
|B008TTPTOQ|Battlefield Bad C...|
|B00DGZDGUM|Nascar The Game 2...|
+----------+--------------------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import desc
# Customers DataFrame
customers_df = digital_video_games_df.groupby("customer_id").agg({"customer_id":"count"})
customers_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   47505356|                 1|
|   21198249|                10|
|   11646226|                 4|
|   50712554|                 1|
|   13490955|                 2|
|    2099668|                 1|
|   29206489|                 1|
|   14231016|                 1|
|   46351928|                 7|
|   28665190|                 1|
|   44172434|                 1|
|   45096613|                 1|
|   13713446|                 8|
|    2863501|                 1|
|   39449017|                 3|
|   21281723|                 5|
|    3953141|                 2|
|    8213600|                 2|
|   33185883|                 1|
|   15109931|                 1|
+-----------+------------------+
only showing top 20 rows



In [8]:
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   21925455|            95|
|   40955995|            92|
|   23564949|            83|
|   52520442|            73|
|   14279279|            72|
|   51520185|            65|
|   20751317|            54|
|   25575022|            49|
|   50529050|            49|
|   11584162|            49|
|   36527466|            48|
|   39942708|            48|
|   31012456|            46|
|   52919259|            45|
|   19371429|            43|
|   46544620|            43|
|   53080901|            42|
|   26151231|            42|
|   45688707|            40|
|   14222166|            39|
+-----------+--------------+
only showing top 20 rows



In [9]:
# Vine DataFrame
vine_df = digital_video_games_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RDHZ0IP0QWF3G|          1|            1|          1|   N|
|R2AQD17WE4BPXV|          1|            1|          1|   N|
|R27V3DHADX5APY|          1|            0|          0|   N|
|R2Q0PGNE0M3ES4|          5|            0|          0|   N|
|R1LB0DS6OZCIMW|          5|            0|          0|   N|
|R3HX7QICC0NOCK|          1|            0|          0|   N|
|R1KXTQXBE0CRQM|          5|            0|          0|   N|
|R39TG3LST9JDA8|          5|            0|          0|   N|
|R1W2UR1JVB2C13|          5|            1|          1|   N|
|R1M0DM7FCXI1H9|          5|            0|          0|   N|
|R2IJD4JDICI4AF|          1|            2|          2|   N|
| RWVPFDHF1M46B|          5|            0|          0|   N|
|R3VRDDUP2Y8GLL|          5|            0|          0|   N|
| RTQ74WPYINEQ4|          1|            

In [10]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/digital_video_games_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)